# https://huggingface.co/docs/transformers/main/zh/llm_tutorial

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

model_id = "mistralai/Mistral-7B-v0.1"

# https://huggingface.co/docs/huggingface_hub/package_reference/environment_variables
# export HF_HOME=$HOME/.hf
# PYTORCH_PRETRAINED_BERT_CACHE = os.getenv("PYTORCH_PRETRAINED_BERT_CACHE", "$HF_HOME/hub")

# With GPU
# device_map确保模型被移动到您的GPU(s)上
# load_in_4bit应用4位动态量化来极大地减少资源需求
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True)
# Without GPU
# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=False)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like mistralai/Mistral-7B-v0.1 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")

model_inputs = tokenizer(["A list of colors: red, blue"],
                         return_tensors="pt").to("cuda")

generated_ids = model.generate(**model_inputs)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

批量输入

In [ ]:
# Most LLMs don't have a pad token by default
tokenizer.pad_token = tokenizer.eos_token
model_inputs = tokenizer(
    ["A list of colors: red, blue", "Portugal is"], return_tensors="pt", padding=True
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

最常见的陷阱列表以及如何避免它们

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Most LLMs don't have a pad token by default
tokenizer.pad_token = tokenizer.eos_token

手动设置max_new_tokens

In [ ]:
model_inputs = tokenizer(["A sequence of numbers: 1, 2"],
                         return_tensors="pt").to("cuda")

# By default, the output will contain up to 20 tokens
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Setting `max_new_tokens` allows you to control the maximum length
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

do_sample=True

In [ ]:
# Set seed or reproducibility -- you don't need this unless you want full reproducibility
from transformers import set_seed
set_seed(42)

model_inputs = tokenizer(["I am a cat."], return_tensors="pt").to("cuda")

# LLM + greedy decoding = repetitive, boring output
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# With sampling, the output becomes more creative!
generated_ids = model.generate(**model_inputs, do_sample=True)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

LLMs是仅解码器架构，意味着它们会持续迭代您的输入提示。如果您的输入长度不相同，则需要对它们进行填充。由于LLMs没有接受过从pad tokens继续训练，因此您的输入需要左填充。确保在生成时不要忘记传递注意力掩码！

In [ ]:
# The tokenizer initialized above has right-padding active by default: the 1st sequence,
# which is shorter, has padding on the right side. Generation fails to capture the logic.
model_inputs = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# With left-padding, it works as expected!
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1", padding_side="left")
# Most LLMs don't have a pad token by default
tokenizer.pad_token = tokenizer.eos_token
model_inputs = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

聊天模板

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceH4/zephyr-7b-alpha", device_map="auto", load_in_4bit=True
)
set_seed(0)
prompt = """How many helicopters can a human eat in one sitting? Reply as a thug."""
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs, max_new_tokens=20)
print(tokenizer.batch_decode(
    generated_ids[:, input_length:], skip_special_tokens=True)[0])
# Oh no, it did not follow our instruction to reply as a thug! Let's see what happens when we write
# a better prompt and use the right template for this model (through `tokenizer.apply_chat_template`)

set_seed(0)
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a thug",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
model_inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=20)
print(tokenizer.batch_decode(
    generated_ids[:, input_length:], skip_special_tokens=True)[0])
# As we can see, it followed a proper thug style 😎